In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [4]:
import sys
sys.path.append("../../") 

from utils.paths import make_dir_line

modality = 'c'
project = 'Introduction to Relational Databases in SQL'
data = make_dir_line(modality, project)

raw = data('raw')

In [5]:
import sqlite3

conn = sqlite3.connect(":memory:")  ## aca se indica el nombre de la db.
cur = conn.cursor()

# 6.4.0_Glue together tables with foreign keys

In [6]:
conn.executescript(
    """
    
    DROP TABLE IF EXISTS university_professors;

    CREATE TABLE university_professors (
        firstname             VARCHAR,
        lastname              VARCHAR,
        university            VARCHAR,
        university_shortname  VARCHAR,
        university_city       VARCHAR,
        function              VARCHAR,
        organization          VARCHAR,
        organization_sector   VARCHAR
    );


    DROP TABLE IF EXISTS professors;

    CREATE TABLE professors (
        firstname            text,
        lastname             text,
        university_shortname text
    );


    DROP TABLE IF EXISTS universities;
    
    CREATE TABLE universities (
        university_shortname text,
        university           text,
        university_city      text
    );


    DROP TABLE IF EXISTS affiliations;

    CREATE TABLE affiliations (
        firstname             text,
        lastname              text,
        function              text
    );


    DROP TABLE IF EXISTS organizations;

    CREATE TABLE organizations (
        organization        text,
        organization_sector text
    );
    
    
    DROP TABLE IF EXISTS cars;

    CREATE TABLE cars (
        make   varchar(64) NOT NULL,
        model  varchar(64) NOT NULL,
        mpg    integer NOT NULL
        
    );
    

    """
)
conn.commit()

In [7]:
df = pd.read_csv(raw / 'university_professors.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
university_professors = list(zip(df.firstname, df.lastname, df.university, df.university_shortname, df.university_city, df.function, df.organization, df.organization_sector))
cur.executemany("INSERT INTO university_professors VALUES  (?,?,?,?,?,?,?,?)", university_professors)

df = pd.read_csv(raw / 'cars.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
cars = list(zip(df.make, df.model, df.mpg))
cur.executemany("INSERT INTO cars VALUES  (?,?,?)", cars)

In [8]:
cur.executescript(
    """
    
    
    INSERT INTO universities 
    SELECT DISTINCT university_shortname, university, university_city 
    FROM university_professors;
    
    
    INSERT INTO professors 
    SELECT DISTINCT firstname, lastname, university_shortname 
    FROM university_professors;


    INSERT INTO affiliations 
    SELECT DISTINCT firstname, lastname, function
    FROM university_professors; 
    
    
    INSERT INTO organizations 
    SELECT DISTINCT organization, organization_sector 
    FROM university_professors; 
    

    """
)
conn.commit()

## 6.4.2 REFERENCE a table with a FOREIGN KEY

In [9]:
# -- Rename the university_shortname column
cur.execute("""
            
            ALTER TABLE professors
            RENAME COLUMN university_shortname TO university_id;

            """).fetchall()

[]

In [10]:
# # -- Add a foreign key on professors referencing universities
# cur.execute("""
            
#             ALTER TABLE professors
#             ADD CONSTRAINT professors_fkey FOREIGN KEY (university_id) REFERENCES universities (id);

#             """).fetchall()

## 6.4.3 Explore foreign key constraints

In [11]:
# -- Try to insert a new professor (fix)
cur.execute("""
            
            INSERT INTO professors (firstname, lastname, university_id)
            VALUES ('Albert', 'Einstein', 'UZH');

            """).fetchall()

[]

In [12]:
cur.execute("""
            


            """).fetchall()

[]

In [13]:
print('Ok_')

Ok_
